# ライブラリ読み込み

In [1]:
# ライブラリのインポート
## データ加工・可視化系ライブラリ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font='IPAexGothic')

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.metrics import classification_report, roc_curve, auc, precision_recall_curve

# scikit-learnのインポートをします
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV


OSError: dlopen(/Users/takumu/VScode/.venv/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <D21A7969-4567-3BC7-94ED-6A9E83AE9D78> /Users/takumu/VScode/.venv/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

# データ読み込み

In [ ]:
# データのインポート
## 学習用データ 13列 11900行、Y列がある
train = pd.read_csv('train.csv')
## テスト用データ 12列 5100行、Y列がない
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
train["education"].unique()

# default, housing, loan　0-1化

In [ ]:
train["default"][train["default"] == "yes"] = 1
train["default"][train["default"] == "no"] = 0
train["default"] = train["default"].astype(int)
train["housing"][train["housing"] == "yes"] = 1
train["housing"][train["housing"] == "no"] = 0
train["housing"] = train["housing"].astype(int)
train["loan"][train["loan"] == "yes"] = 1
train["loan"][train["loan"] == "no"] = 0
train["loan"] = train["loan"].astype(int)

In [ ]:
train.info()

# dummy変数

In [ ]:
train_dummy = pd.get_dummies(train)
train_dummy.head()

# 標準化(age,balance,day,duration,campaign,pdays)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_dummy["age_std"] = scaler.fit_transform(train_dummy[["age"]])
train_dummy["balance_std"] = scaler.fit_transform(train_dummy[["balance"]])
train_dummy["day_std"] = scaler.fit_transform(train_dummy[["day"]])
train_dummy["duration_std"] = scaler.fit_transform(train_dummy[["duration"]])
train_dummy["campaign_std"] = scaler.fit_transform(train_dummy[["campaign"]])
train_dummy["pdays_std"] = scaler.fit_transform(train_dummy[["pdays"]])
train_dummy["previous_std"] = scaler.fit_transform(train_dummy[["previous"]])
train_dummy_std = train_dummy.drop(["age", "balance", "day", "duration", "campaign", "pdays", "previous"], axis=1)


# 相関行列

In [ ]:
corr_mat = train_dummy_std.corr(method='pearson')
corr_mat

In [ ]:
sns.heatmap(corr_mat,
            vmin=-1.0,
            vmax=1.0,
            center=0,
            #annot=True, # True:格子の中に値を表示
            fmt='.1f',
            xticklabels=corr_mat.columns.values,
            yticklabels=corr_mat.columns.values,
            cmap='coolwarm'
           )
plt.show()

# test_data加工

In [ ]:
test["default"][test["default"] == "yes"] = 1
test["default"][test["default"] == "no"] = 0
test["default"] = test["default"].astype(int)
test["housing"][test["housing"] == "yes"] = 1
test["housing"][test["housing"] == "no"] = 0
test["housing"] = test["housing"].astype(int)
test["loan"][test["loan"] == "yes"] = 1
test["loan"][test["loan"] == "no"] = 0
test["loan"] = test["loan"].astype(int)

test_dummy = pd.get_dummies(test)

test_dummy["age_std"] = scaler.fit_transform(test_dummy[["age"]])
test_dummy["balance_std"] = scaler.fit_transform(test_dummy[["balance"]])
test_dummy["day_std"] = scaler.fit_transform(test_dummy[["day"]])
test_dummy["duration_std"] = scaler.fit_transform(test_dummy[["duration"]])
test_dummy["campaign_std"] = scaler.fit_transform(test_dummy[["campaign"]])
test_dummy["pdays_std"] = scaler.fit_transform(test_dummy[["pdays"]])
test_dummy["previous_std"] = scaler.fit_transform(test_dummy[["previous"]])
test_dummy_std = test_dummy.drop(["job_unknown","age", "balance", "day", "duration", "campaign", "pdays", "previous"], axis=1)


# 学習(決定木)

In [ ]:
# scikit-learnのインポートをします
from sklearn import tree

In [ ]:
# 「train」の目的変数と説明変数の値を取得
target = train_dummy_std["y"].values
features = train_dummy_std.drop(["y"], axis=1).values
# 「test」の説明変数の値を取得
test_features = test_dummy_std.values
 
# 決定木の作成とアーギュメントの設定
max_depth = 100
min_samples_split = 2
my_tree_two = tree.DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split, random_state = 1)
result = my_tree_two
my_tree_two = my_tree_two.fit(features, target) 
 

# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction = my_tree_two.predict(test_features)
result.score(features,target)

In [ ]:
# PassengerIdを取得
PassengerId = np.array(test["id"]).astype(int)
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId)
display(my_solution.head())
 
# my_tree_one.csvとして書き出し
my_solution.to_csv("my_tree_one.csv",header=False)

# 学習(ランダムフォレスト)

In [ ]:
# 決定木の作成とアーギュメントの設定
max_depth= 10
min_samples_split= 2 
n_estimators=50
my_tree_two = RandomForestClassifier(max_depth = max_depth, min_samples_split = min_samples_split,n_estimators = n_estimators, random_state = 1)
result = my_tree_two
my_tree_two = my_tree_two.fit(features, target) 
 

# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction = my_tree_two.predict(test_features)
result.score(features,target)

# 学習(ロジスティック分類)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
result = lr
logistic = lr.fit(features, target) 
 

# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction = logistic.predict(test_features)
result.score(features,target)

In [ ]:
# PassengerIdを取得
PassengerId = np.array(test["id"]).astype(int)
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId)
display(my_solution.head())
 
# my_tree_one.csvとして書き出し
my_solution.to_csv("my_logistic.csv",header=False)

# 学習(SVM)

In [ ]:
from sklearn.svm import SVC

C = 5
kernel = 'rbf'
gamma = 1
clf = SVC(C=C, kernel=kernel, gamma=gamma)
svm = clf.fit(features, target) 
 

# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction = svm.predict(test_features)
clf.score(features,target)

In [ ]:
# PassengerIdを取得
PassengerId = np.array(test["id"]).astype(int)
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId)
display(my_solution.head())
 
# my_tree_one.csvとして書き出し
my_solution.to_csv("my_svm.csv",header=False)

# 学習(LightGBM)

In [ ]:
import lightgbm as lgb

kf  = KFold(n_splits=5, shuffle=True, random_state=1234)
accuracies = []

for train_index, val_index in kf.split(features):
    # モデルと特徴選択器を新たに作成
    clf = lgb.LGBMClassifier()

    # データ分割
    X_train, X_val = features[train_index], features[val_index]
    y_train, y_val = target[train_index], target[val_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_val)

    # 精度の計算と保存
    accuracy = accuracy_score(y_val, predictions)
    accuracies.append(accuracy)
    print('Accuracy: %.2f' % accuracy)

# 平均精度の計算
average_accuracy = sum(accuracies) / len(accuracies)
print('Average Accuracy: %.2f' % average_accuracy)

# 交差検証(KFold)

In [ ]:
kf  = KFold(n_splits=5, shuffle=True, random_state=1234)
accuracies = []

for train_index, val_index in kf.split(features):
    # モデルと特徴選択器を新たに作成
    model = RandomForestClassifier(n_estimators=100)

    # データ分割
    X_train, X_val = features[train_index], features[val_index]
    y_train, y_val = target[train_index], target[val_index]

    model.fit(X_train, y_train)
    predictions = model.predict(X_val)

    # 精度の計算と保存
    accuracy = accuracy_score(y_val, predictions)
    accuracies.append(accuracy)
    print('Accuracy: %.2f' % accuracy)

# 平均精度の計算
average_accuracy = sum(accuracies) / len(accuracies)
print('Average Accuracy: %.2f' % average_accuracy)

# グリッドサーチ

In [ ]:
from sklearn.model_selection import GridSearchCV
# ランダムフォレストモデルのインスタンス化
model = RandomForestClassifier(random_state=1234)

# ハイパーパラメータのグリッドを定義
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCVの設定
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy',verbose=2)

# グリッドサーチの実行
grid_search.fit(features, target)

# 結果の表示
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)